# IMPORT LIBRARIES

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# **Loading the dataset**

In [ ]:
(train_data, test_data), ds_info = tfds.load('kmnist', split=['train', 'test'], shuffle_files=True, as_supervised=True, with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/60000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/kmnist/incomplete.5QZ6JI_3.0.1/kmnist-train.tfrecord*...:   0%|          |…

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/kmnist/incomplete.5QZ6JI_3.0.1/kmnist-test.tfrecord*...:   0%|          | …

Dataset kmnist downloaded and prepared to /root/tensorflow_datasets/kmnist/3.0.1. Subsequent calls will reuse this data.


# **Preprocess the data**

In [ ]:
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

(train_data, test_data), ds_info = tfds.load('kmnist', split=['train', 'test'], shuffle_files=True, as_supervised=True, with_info=True)

train_data = train_data.map(preprocess).batch(128).prefetch(tf.data.AUTOTUNE)
test_data = test_data.map(preprocess).batch(128).prefetch(tf.data.AUTOTUNE)

**Convert dataset to numpy arrays**

In [ ]:
def dataset_to_numpy(dataset):
    images, labels = [], []
    for image_batch, label_batch in dataset:
        images.append(image_batch.numpy())
        labels.append(label_batch.numpy())
    return np.concatenate(images), np.concatenate(labels)

X_train, y_train = dataset_to_numpy(train_data)
X_test, y_test = dataset_to_numpy(test_data)

Reshape the images

In [ ]:
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

# **Radial Basis Function (RBF) Network**

In [ ]:
class RBFLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs, centers):
        super(RBFLayer, self).__init__()
        self.num_outputs = num_outputs
        self.centers = centers

    def build(self, input_shape):
        self.betas = self.add_weight(name='betas', shape=(self.num_outputs,),
                                     initializer='ones', trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        C = tf.expand_dims(self.centers, axis=0)
        H = tf.transpose(C - tf.expand_dims(inputs, axis=1), perm=[0, 2, 1])
        return tf.exp(-self.betas * tf.reduce_sum(H**2, axis=1))

#**Training**

** Using K-means to determine RBF centers**

In [ ]:
num_rbf = 100
kmeans = KMeans(n_clusters=num_rbf, n_init=10)
kmeans.fit(X_train)
rbf_centers = kmeans.cluster_centers_

**Building the model**

In [ ]:
model = tf.keras.Sequential([
    RBFLayer(num_rbf, rbf_centers),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

**Train the model**

In [ ]:
history = model.fit(X_train, y_train, epochs=100, validation_split=0.2, verbose=1)

Epoch 1/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 29s 18ms/step - accuracy: 0.1013 - loss: 2.3037 - val_accuracy: 0.1032 - val_loss: 2.3033
Epoch 2/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 27s 18ms/step - accuracy: 0.1046 - loss: 2.3015 - val_accuracy: 0.1037 - val_loss: 2.3039
Epoch 3/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 49s 23ms/step - accuracy: 0.1043 - loss: 2.3160 - val_accuracy: 0.1018 - val_loss: 2.3028
Epoch 4/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 26s 18ms/step - accuracy: 0.1354 - loss: 2.2506 - val_accuracy: 0.1797 - val_loss: 2.1537
Epoch 5/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 45s 20ms/step - accuracy: 0.1800 - loss: 2.1603 - val_accuracy: 0.1809 - val_loss: 2.1309
Epoch 6/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 40s 20ms/step - accuracy: 0.1846 - loss: 2.1550 - val_accuracy: 0.1927 - val_loss: 2.1349
Epoch 7/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 41s 20ms/step - accuracy: 0.1878 - loss: 2.1552 - val_accuracy: 0.2282 - val_loss: 2.0390
Epoch 8/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 40s 19ms/step - accuracy: 

# **Evaluation**

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.4f}")

In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
conf_mat = confusion_matrix(y_test, y_pred_classes)

Visualizing the Confusion Matrix

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

Visualizing the training history

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# **Analysis**

In [ ]:
print("Analysis:")
print("Strengths of RBF Network:")
print("1. Can model complex non-linear decision boundaries")
print("2. Faster training compared to some other neural network architectures")
print("3. Good at handling localized features in the input space")

print("\nLimitations of RBF Network:")
print("1. Performance heavily depends on the choice of centers")
print("2. May require a large number of RBF units for complex problems")
print("3. Can be sensitive to the scale of the input features")

print("\nEffect of number of RBF units:")
print("Increasing the number of RBF units can improve the model's ability to capture complex patterns,")
print("but it also increases computational cost and may lead to overfitting if too many units are used.")
print("Finding the optimal number often requires experimentation and cross-validation.")